### Dependencias necesarias  

In [2]:
# pip install tensorflow tensorflow-hub librosa numpy

### Importando las dependencias

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
import os

### Configurando Constantes con las direcciones

In [11]:

# URLs de los modelos
YAMNET_MODEL_URL = "https://tfhub.dev/google/yamnet/1"
VGGISH_MODEL_URL = "https://tfhub.dev/google/vggish/1"

# Nombre del archivo de audio local
AUDIO_FILE = "guitar-01.mp3"

In [12]:
# Índices de las clases de guitarra en YAMNet : URL : https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/yamnet_class_map.csv
GUITAR_CLASSES = [132, 425]  # Acoustic guitar, Electric guitar

### Creando las funciones para reusar

In [8]:
# Función para cargar el audio y convertirlo a la frecuencia de muestreo necesaria
def load_audio_file(file_path, target_sample_rate=16000):
    audio, sr = librosa.load(file_path, sr=target_sample_rate)
    # Normalizar el audio entre -1 y 1
    audio = audio / np.max(np.abs(audio))
    return audio, sr

# Cargar el modelo de YAMNet
def load_yamnet_model():
    yamnet_model = hub.load(YAMNET_MODEL_URL)
    return yamnet_model

# Cargar el modelo de VGGish
def load_vggish_model():
    vggish_model = hub.load(VGGISH_MODEL_URL)
    return vggish_model

# Realizar la predicción con YAMNet
def predict_yamnet(yamnet_model, audio_data):
    # Asegurarse de que el audio sea un tensor de float32
    audio_tensor = tf.convert_to_tensor(audio_data, dtype=tf.float32)
    scores, embeddings, spectrogram = yamnet_model(audio_tensor)
    predicted_class = tf.argmax(scores, axis=-1)
    return predicted_class.numpy(), scores.numpy()

# Realizar la predicción con VGGish
def predict_vggish(vggish_model, audio_data):
    # VGGish solo necesita embeddings, por lo que no se necesita predicción directa
    audio_tensor = tf.convert_to_tensor(audio_data, dtype=tf.float32)
    embeddings = vggish_model(audio_tensor)
    return embeddings.numpy()

### Cargando los modelos

In [9]:
# Preparar el audio para la entrada del modelo
audio_data, sample_rate = load_audio_file(AUDIO_FILE)

# Cargar modelos
yamnet_model = load_yamnet_model()
vggish_model = load_vggish_model()

### Realizando predicciones

In [10]:
# Realizar predicciones con YAMNet
yamnet_pred_class, yamnet_scores = predict_yamnet(yamnet_model, audio_data)

# Realizar predicciones con VGGish (usando la misma entrada)
vggish_embeddings = predict_vggish(vggish_model, audio_data)

# Mostrar resultados
print("YAMNet - Predicted Class:", yamnet_pred_class)
print("YAMNet - Scores Shape:", yamnet_scores.shape)
print("VGGish - Embeddings Shape:", vggish_embeddings.shape)


YAMNet - Predicted Class: [132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132
 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132
 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132 132
 132 132 132 132 132 132 132 132 132 132]
YAMNet - Scores Shape: (64, 521)
VGGish - Embeddings Shape: (32, 128)


In [14]:
print("VGGish - Embeddings Shape:", vggish_embeddings)


VGGish - Embeddings Shape: [[-0.7336062  -0.18590459  0.39883107 ... -0.85758007 -0.07540295
  -0.2769341 ]
 [-1.085657   -0.51505184  0.36990386 ... -1.0317211   0.16072503
  -0.17959717]
 [-0.88073695 -0.49706617  0.00116414 ... -0.7746079  -0.19385038
  -0.34082073]
 ...
 [-0.5604361  -0.36990973  0.15180324 ... -0.7322267  -0.1039039
  -0.21786481]
 [-0.9799707  -0.4949585   0.21475768 ... -1.1171416  -0.18682863
  -0.33782995]
 [-0.79963374 -0.43110666  0.2771427  ... -0.95338655 -0.28829026
  -0.30207974]]


In [13]:
# Identificar si alguna de las clases predichas es una guitarra
is_guitar = any(cls in GUITAR_CLASSES for cls in yamnet_pred_class)

# Mostrar resultados
if is_guitar:
    print("El audio contiene una guitarra.")
else:
    print("El audio no contiene una guitarra.")

El audio contiene una guitarra.
